In [1]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaTokenizerFast
from datasets import Dataset

In [2]:
def read_even_lines(directory, is_v):
    data = ""
    with open(directory, 'r') as file:
        lines = file.readlines()[1::2]  # Read only even lines
        data += "".join(lines) + "\n"
    return {'DNA': data, 'is_v': is_v}

In [3]:
training_data_paths = ["../data/training/host_tr.fa", 
                       "../data/training/virus_tr.fa"]

valing_data_paths = ["../data/training/host_val.fa", 
                      "../data/training/virus_val.fa"]

In [4]:
tokenizer = ByteLevelBPETokenizer()
tokenizer = RobertaTokenizerFast.from_pretrained("../_1_tokenizer/BPETokenizer", max_len=512)

tokenized_train_data = []
tokenized_val_data = []

for i, path in enumerate(training_data_paths):
    is_v = 1 if i % 2 != 0 else 0  # Assign 1 for virus files and 0 for host files
    even_lines_data = read_even_lines(path, is_v)
    tokenized_lines = tokenizer(even_lines_data['DNA'], padding='max_length', truncation=True, max_length=512)
    tokenized_train_data.append({'DNA': tokenized_lines['input_ids'], 'is_v': is_v})

for i, path in enumerate(valing_data_paths):
    is_v = 1 if i % 2 != 0 else 0  # Assign 1 for virus files and 0 for host files
    even_lines_data = read_even_lines(path, is_v)
    tokenized_lines = tokenizer(even_lines_data['DNA'], padding='max_length', truncation=True, max_length=512)
    tokenized_val_data.append({'DNA': tokenized_lines['input_ids'], 'is_v': is_v})

train_dataset = Dataset.from_dict({'DNA': [item['DNA'] for item in tokenized_train_data], 'is_v': [item['is_v'] for item in tokenized_train_data]})
val_dataset = Dataset.from_dict({'DNA': [item['DNA'] for item in tokenized_val_data], 'is_v': [item['is_v'] for item in tokenized_val_data]})

train_dataset.save_to_disk('Tokenized_train_dataset.hf')
val_dataset.save_to_disk('Tokenized_val_dataset.hf')

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]